# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it.

You will find more info about virtual environments in the course content and on the web.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [450]:
# import the requests library (1 line)

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)

# assign the /status endpoint to another variable called status_url (1 line)

# query the /status endpoint using the get() method and store it in the req variable (1 line)

# check the status_code using a condition and print appropriate messages (4 lines)
import requests
root_url = " https://country-leaders.onrender.com/"
status_url = "status"
req = requests.get(f"{root_url}/{status_url}")
if req.status_code == 200:
    print(f"its ok:{req.text}")
else:
    print("responcse.status_code")




its ok:"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [451]:
# Set the countries_url variable (1 line)

# query the /countries endpoint using the get() method and store it in the req variable (1 line)

# Get the JSON content and store it in the countries variable (1 line)

# display the request's status code and the countries variable (1 line)
countries_url = "countries"
req = requests.get(f"{root_url}/{countries_url}")
countries = req.json()
print(countries)


{'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [452]:
# Set the cookie_url variable (1 line)

# Query the enpoint, set the cookies variable and display it (2 lines)
cookie_url = "cookie"
cookie = requests.get(f"{root_url}/{cookie_url}").cookies
print(cookie)



<RequestsCookieJar[<Cookie user_cookie=913d2fe0-b2a9-43bf-b7ea-82cfc16ee5a0 for country-leaders.onrender.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [453]:
# query the /countries endpoint, assign the output to the countries variable (1 line)

# display the countries variable (1 line)
countries_url = f"{root_url}/countries"
countries = requests.get(countries_url, cookies=cookie).json()
print(countries)


['us', 'be', 'ru', 'ma', 'fr']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [454]:
# Set the leaders_url variable (1 line)

# query the /leaders endpoint, assign the output to the leaders variable (1 line)

# display the leaders variable (1 line)
leaders_url = f"{root_url}/leaders"
leaders = requests.get(leaders_url, cookies=cookie).json()
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [455]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
# assign the output to the leaders variable (1 line)

# display the leaders variable (1 line)
leader = requests.get(leaders_url, cookies=cookie, params={"country":'countries'}).json()
print(leader)

{'message': "This country is not covered by the API. Please pick one among ['us', 'be', 'ru', 'ma', 'fr']"}


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [456]:
# 4 lines
leaders_per_country = {}
for country in countries:
    leaders_url = f"{root_url}/leaders"
    leader = requests.get(leaders_url, cookies=cookie, params={"country": country})
    leaders_per_country[country] = leader.json()
print(leaders_per_country)
    



{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [457]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [458]:
# < 15 lines
def get_leaders():
    # 1,define the urls
    leaders_url = f"{root_url}/leaders"
    countries_url = f"{root_url}/countries"
    cookie_url = f"{root_url}/cookie"

    # 2, Get the cookies
    cookies = requests.get(cookie_url).cookies.get_dict()

    # 3 Get the countries
    countries = requests.get(countries_url, cookies=cookies).json()

    # 4 Dictionary to hold leaders per country
    leaders_per_country = {}
    for country in countries:
        leaders_response = requests.get(leaders_url, cookies=cookies, params={'country': country}).json()
        leaders_per_country[country] = leaders_response
    # 5 return the dictionary
    return leaders_per_country

    


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [459]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [480]:
# 3 lines
first_leader = next(iter(leaders_per_country.values()))[0]
wiki_url = first_leader.get("wikipedia_url")
wiki_req = requests.get(wiki_url)
print(wiki_req.text)

<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" lang="en" dir="ltr">
<head>
<meta charset="UTF-8">
<title>George Washington - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [461]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_req.text, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   George Washington - Wikipedia
  </title>
  <script>
   (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [462]:
# 2 line
paragraphs = soup.find_all('p')
print(paragraphs)

[<p class="mw-empty-elt">
</p>, <p><b>George Washington</b> (February 22, 1732 – <span class="nowrap">December 14</span>, 1799) was a <a class="mw-redirect" href="/wiki/Founding_Father_of_the_United_States" title="Founding Father of the United States">Founding Father of the United States</a>, military officer, and farmer who served as the first <a href="/wiki/President_of_the_United_States" title="President of the United States">president of the United States</a> from 1789 to 1797. Appointed by the <a href="/wiki/Second_Continental_Congress" title="Second Continental Congress">Second Continental Congress</a> as commander of the <a href="/wiki/Continental_Army" title="Continental Army">Continental Army</a> in 1775, Washington led <a href="/wiki/Patriot_(American_Revolution)" title="Patriot (American Revolution)">Patriot</a> forces to victory in the <a href="/wiki/American_Revolutionary_War" title="American Revolutionary War">American Revolutionary War</a>. He then served as president of

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [463]:
# <10 lines
first_paragraph = None
for paragraph in paragraphs:
    text = paragraph.get_text().strip()
    if text:  # Check if the paragraph is not empty
        first_paragraph = text
        break
print(first_paragraph)


George Washington (February 22, 1732 – December 14, 1799) was a Founding Father of the United States, military officer, and farmer who served as the first president of the United States from 1789 to 1797. Appointed by the Second Continental Congress as commander of the Continental Army in 1775, Washington led Patriot forces to victory in the American Revolutionary War. He then served as president of the Constitutional Convention in 1787, which drafted the current Constitution of the United States. Washington has thus become commonly known as the "Father of His Country".


At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [464]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)        
    wiki_req = requests.get(wikipedia_url)   
    soup = BeautifulSoup(wiki_req.text, 'html.parser')
    paragraphs = soup.find_all('p')    
    first_paragraph = None
    for paragraph in paragraphs:
        first_paragraph = paragraph.get_text(strip=True)  
    return first_paragraph

In [465]:
# Test: 3 lines
first_paragraph = get_first_paragraph(wiki_url)
print(first_paragraph)

https://en.wikipedia.org/wiki/George_Washington
Washington appears on contemporary U.S. currency, including theone-dollar bill, thePresidential one-dollar coinand thequarter-dollar coin(theWashington quarter). Washington andBenjamin Franklinappeared on thenation's first postage stampsin 1847. Washington has since appeared on many postage issues, more than any other person.[456]


### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [466]:
# 3 lines
import re
#remove emultiple spaces and newlines
first_paragraph = re.sub(r'\s+', ' ', first_paragraph)
#ensure space between lowercase and upper case
first_paragraph = re.sub(r'([a-z])([A-Z])', r'\1 \2', first_paragraph)
print(first_paragraph)




Washington appears on contemporary U.S. currency, including theone-dollar bill, the Presidential one-dollar coinand thequarter-dollar coin(the Washington quarter). Washington and Benjamin Franklinappeared on thenation's first postage stampsin 1847. Washington has since appeared on many postage issues, more than any other person.[456]


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [467]:
# 10 lines

def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)        
    wiki_req = requests.get(wikipedia_url)   
    soup = BeautifulSoup(wiki_req.text, 'html.parser')
    paragraphs = soup.find_all('p')    
    first_paragraph = None
    for paragraph in paragraphs:
        first_paragraph = paragraph.get_text(strip=True)  
    return first_paragraph
first_paragraph = get_first_paragraph(wiki_url)
print(first_paragraph)

https://en.wikipedia.org/wiki/George_Washington
Washington appears on contemporary U.S. currency, including theone-dollar bill, thePresidential one-dollar coinand thequarter-dollar coin(theWashington quarter). Washington andBenjamin Franklinappeared on thenation's first postage stampsin 1847. Washington has since appeared on many postage issues, more than any other person.[456]


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [468]:
# < 20 lines



## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [469]:
# < 20 lines

In [470]:
# Check the output of your function (2 lines)


Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [471]:
# < 25 lines



Check the output of your function again.

In [472]:
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [473]:
# < 20 lines


Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [474]:
# <25 lines


Test your new functions.



## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [475]:
# 3 lines


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [476]:
# 3 lines


Make a function `save(leaders_per_country)` to call this code easily.

In [477]:
# 3 lines


In [478]:
# Call the function (1 line)


## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!